In [1]:
import numpy as np
import pandas as pd

# Efficacy Test

In [2]:
# Data Loading
fakeData_arr = pd.read_excel('RCT_fakeData.xlsx').to_numpy()

y = fakeData_arr[:, 2]
X = np.concatenate((np.ones((y.shape[0], 1)), fakeData_arr[:, 1].reshape(-1, 1)), axis=1)
coeff = np.linalg.lstsq(X, y, rcond=None)[0]
residual = y - np.dot(X, coeff)

# comparison of results
print("Beta (the value getting from least square method): ", coeff[1])   # Beta
t_grp_outcome = np.mean(fakeData_arr[fakeData_arr[:, 1] == 1][:, 2]) # Treatment group outcome
c_grp_outcome = np.mean(fakeData_arr[fakeData_arr[:, 1] == 0][:, 2]) # Control group outcome
print( "Beta (the value getting from calculation directly): ", t_grp_outcome - c_grp_outcome ) # Beta

N_treated = len(fakeData_arr[fakeData_arr[:, 1] == 1])
N = len(fakeData_arr[:, 1])
N_controlled = N - N_treated

tStat = np.sqrt(N) * coeff[1] / np.sqrt( (N/N_treated + N/N_controlled) * np.var(residual, ddof=1) )
print("T-stat : ", tStat)

Beta (the value getting from least square method):  -0.09062586032248597
Beta (the value getting from calculation directly):  -0.09062586032248535
T-stat :  -24.982185711016047


# Check the result with OLS Package

In [3]:
import statsmodels.api as sm
fakeData_df = pd.read_excel('RCT_fakeData.xlsx')

In [4]:
X0 = fakeData_df['vaccineTreatment']
X = sm.add_constant(X0)
Y = fakeData_df['CoronaPositive']

In [5]:
model = sm.OLS(Y, X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         CoronaPositive   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                  0.020
Method:                 Least Squares   F-statistic:                     624.1
Date:                Sun, 03 Oct 2021   Prob (F-statistic):          2.40e-136
Time:                        18:11:20   Log-Likelihood:                -5215.9
No. Observations:               30000   AIC:                         1.044e+04
Df Residuals:                   29998   BIC:                         1.045e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.1205      0.002     60.658      0.000       0.117       0.124
vaccineTreatment    -0.0906      0.004    -24.982      0.000      -0.098      -0.084
==============================================================================
Omnibus:                    15742.167   Durbin-Watson:                   1.998
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            74589.322
Skew:                           2.702   Prob(JB):                         0.00
Kurtosis:                       8.520   Cond. No.                         2.42
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""